In [24]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, count, lit,to_date,date_diff


In [25]:
spark=SparkSession.builder.appName("regression").getOrCreate()
data_path="file:///home/siwenyu/桌面/Spark-lab4/res/task1_1/part-00000-3a410553-5280-4da6-8b49-741df2889028-c000.csv"
df=spark.read.csv(data_path,header=True)
df.show()

24/12/23 19:56:13 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


+-----------+------------------+----------------+
|report_date|total_purchase_amt|total_redeem_amt|
+-----------+------------------+----------------+
|   20140824|         130195484|       191080151|
|   20140812|         258493673|       309754858|
|   20140728|         371762756|       345986909|
|   20140716|         394890140|       234775948|
|   20140704|         211649838|       264494550|
|   20140320|         365011495|       336076380|
|   20140404|         251895894|       200192637|
|   20140416|         387847838|       255914640|
|   20140428|         324937272|       327724735|
|   20140512|         325108597|       293952908|
|   20140524|         160073254|       154409868|
|   20140608|         302171269|       169525332|
|   20140620|         251582530|       286583065|
|   20131020|          47766681|        50884342|
|   20131104|         300027403|       130970051|
|   20131116|         118085705|        28996272|
|   20131128|         139760425|        61453591|


In [26]:
from pyspark.sql import functions as F

# preprocess
# date conversion to yyMMdd, but first select a base
base_date = F.lit("2013-07-01").cast("date")
df = df.withColumn("report_date", F.to_date(col("report_date"), "yyyyMMdd"))
# sequence based on base
df = df.withColumn("sequence", F.datediff(col("report_date"), base_date))
df.show()



+-----------+------------------+----------------+--------+
|report_date|total_purchase_amt|total_redeem_amt|sequence|
+-----------+------------------+----------------+--------+
| 2014-08-24|         130195484|       191080151|     419|
| 2014-08-12|         258493673|       309754858|     407|
| 2014-07-28|         371762756|       345986909|     392|
| 2014-07-16|         394890140|       234775948|     380|
| 2014-07-04|         211649838|       264494550|     368|
| 2014-03-20|         365011495|       336076380|     262|
| 2014-04-04|         251895894|       200192637|     277|
| 2014-04-16|         387847838|       255914640|     289|
| 2014-04-28|         324937272|       327724735|     301|
| 2014-05-12|         325108597|       293952908|     315|
| 2014-05-24|         160073254|       154409868|     327|
| 2014-06-08|         302171269|       169525332|     342|
| 2014-06-20|         251582530|       286583065|     354|
| 2013-10-20|          47766681|        50884342|     11

In [27]:
# only select columns: sequence, total_purchase_amt; sequence, total_redeem_amt, 2 dataframes
purchase = df.select("sequence", "total_purchase_amt").orderBy("sequence")
redeem = df.select("sequence", "total_redeem_amt").orderBy("sequence")
purchase.show()
redeem.show()

+--------+------------------+
|sequence|total_purchase_amt|
+--------+------------------+
|       0|          32488348|
|       1|          29037390|
|       2|          27270770|
|       3|          18321185|
|       4|          11648749|
|       5|          36751272|
|       6|           8962232|
|       7|          57258266|
|       8|          26798941|
|       9|          30696506|
|      10|          44075197|
|      11|          34183904|
|      12|          15164717|
|      13|          22615303|
|      14|          48128555|
|      15|          50622847|
|      16|          29015682|
|      17|          24234505|
|      18|          33680124|
|      19|          20439079|
+--------+------------------+
only showing top 20 rows

+--------+----------------+
|sequence|total_redeem_amt|
+--------+----------------+
|       0|         5525022|
|       1|         2554548|
|       2|         5953867|
|       3|         6410729|
|       4|         2763587|
|       5|         1616635|
| 

In [28]:
# preprocess for isotonic regression

# check the null values
# purchase.where(col("total_purchase_amt").isNull()).show()
# redeem.where(col("total_redeem_amt").isNull()).show()

# transfer amt to double
purchase = purchase.withColumn("total_purchase_amt", col("total_purchase_amt").cast("double"))
redeem = redeem.withColumn("total_redeem_amt", col("total_redeem_amt").cast("double"))
# purchase.show()
# redeem.show()

# select "features" and "labels" column for regression

from pyspark.ml.feature import VectorAssembler
def preprocess(df, val_col):
    assembler=VectorAssembler(inputCols=["sequence"],outputCol="features")
    df = assembler.transform(df).select("features", col(val_col).alias("label"))
    return df

purchase = preprocess(purchase, "total_purchase_amt")
redeem = preprocess(redeem, "total_redeem_amt")
purchase.show()
redeem.show()


+--------+-----------+
|features|      label|
+--------+-----------+
|   [0.0]|3.2488348E7|
|   [1.0]| 2.903739E7|
|   [2.0]| 2.727077E7|
|   [3.0]|1.8321185E7|
|   [4.0]|1.1648749E7|
|   [5.0]|3.6751272E7|
|   [6.0]|  8962232.0|
|   [7.0]|5.7258266E7|
|   [8.0]|2.6798941E7|
|   [9.0]|3.0696506E7|
|  [10.0]|4.4075197E7|
|  [11.0]|3.4183904E7|
|  [12.0]|1.5164717E7|
|  [13.0]|2.2615303E7|
|  [14.0]|4.8128555E7|
|  [15.0]|5.0622847E7|
|  [16.0]|2.9015682E7|
|  [17.0]|2.4234505E7|
|  [18.0]|3.3680124E7|
|  [19.0]|2.0439079E7|
+--------+-----------+
only showing top 20 rows

+--------+-----------+
|features|      label|
+--------+-----------+
|   [0.0]|  5525022.0|
|   [1.0]|  2554548.0|
|   [2.0]|  5953867.0|
|   [3.0]|  6410729.0|
|   [4.0]|  2763587.0|
|   [5.0]|  1616635.0|
|   [6.0]|  3982735.0|
|   [7.0]|  8347729.0|
|   [8.0]|  3473059.0|
|   [9.0]|  2597169.0|
|  [10.0]|  3508800.0|
|  [11.0]|  8492573.0|
|  [12.0]|  3482829.0|
|  [13.0]|  2784107.0|
|  [14.0]|1.3107943E7|
|  [15.0

In [107]:
# train isotonic regression model


# predict

from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import IsotonicRegression, GBTRegressor, RandomForestRegressor, GeneralizedLinearRegression
# iso_reg=IsotonicRegression(featuresCol="features",labelCol="label")
# purchase_model=iso_reg.fit(purchase)
# redeem_model=iso_reg.fit(redeem)

# gbt=GBTRegressor(featuresCol="features",labelCol="label",maxIter=100,seed=1,stepSize=0.1,maxDepth=7)
# purchase_model=gbt.fit(purchase)
# redeem_model=gbt.fit(redeem)

# rf=RandomForestRegressor(featuresCol="features",labelCol="label",maxDepth=7,numTrees=100,seed=1)
# purchase_model=rf.fit(purchase)
# redeem_model=rf.fit(redeem)

glr=GeneralizedLinearRegression(featuresCol="features",labelCol="label",family="gaussian",maxIter=520,regParam=0.233,tol=1e-7,link="log",linkPower=3.3)
purchase_model=glr.fit(purchase)
redeem_model=glr.fit(redeem)

# predict the next 30 days

last_day=purchase.select("features").orderBy(col("features").desc()).first()[0]
# cast last_day to int
# last_day=last_day.astype(int)
print(last_day)

future_days=[(last_day+i,) for i in range(1,31)]
print(future_days)


future_days_df=spark.createDataFrame(future_days,["sequence"])
assembler=VectorAssembler(inputCols=["sequence"],outputCol="features")
future_days_df=assembler.transform(future_days_df)

purchase_future=purchase_model.transform(future_days_df)
redeem_future=redeem_model.transform(future_days_df)


purchase_future.show()
redeem_future.show()



24/12/23 21:55:40 WARN GeneralizedLinearRegression: When family is not tweedie, use param link to specify link function. Setting param linkPower will take no effect.
24/12/23 21:55:43 WARN GeneralizedLinearRegressionModel: When family is not tweedie, use param link to specify link function. Setting param linkPower will take no effect.
24/12/23 21:55:43 WARN GeneralizedLinearRegressionModel: When family is not tweedie, use param link to specify link function. Setting param linkPower will take no effect.
24/12/23 21:55:43 WARN GeneralizedLinearRegression: When family is not tweedie, use param link to specify link function. Setting param linkPower will take no effect.
24/12/23 21:55:45 WARN GeneralizedLinearRegressionModel: When family is not tweedie, use param link to specify link function. Setting param linkPower will take no effect.
24/12/23 21:55:45 WARN GeneralizedLinearRegressionModel: When family is not tweedie, use param link to specify link function. Setting param linkPower will 

[426.0]
[(DenseVector([427.0]),), (DenseVector([428.0]),), (DenseVector([429.0]),), (DenseVector([430.0]),), (DenseVector([431.0]),), (DenseVector([432.0]),), (DenseVector([433.0]),), (DenseVector([434.0]),), (DenseVector([435.0]),), (DenseVector([436.0]),), (DenseVector([437.0]),), (DenseVector([438.0]),), (DenseVector([439.0]),), (DenseVector([440.0]),), (DenseVector([441.0]),), (DenseVector([442.0]),), (DenseVector([443.0]),), (DenseVector([444.0]),), (DenseVector([445.0]),), (DenseVector([446.0]),), (DenseVector([447.0]),), (DenseVector([448.0]),), (DenseVector([449.0]),), (DenseVector([450.0]),), (DenseVector([451.0]),), (DenseVector([452.0]),), (DenseVector([453.0]),), (DenseVector([454.0]),), (DenseVector([455.0]),), (DenseVector([456.0]),)]


24/12/23 21:55:46 WARN GeneralizedLinearRegressionModel: When family is not tweedie, use param link to specify link function. Setting param linkPower will take no effect.
24/12/23 21:55:46 WARN GeneralizedLinearRegressionModel: When family is not tweedie, use param link to specify link function. Setting param linkPower will take no effect.
24/12/23 21:55:46 WARN GeneralizedLinearRegressionModel: When family is not tweedie, use param link to specify link function. Setting param linkPower will take no effect.
24/12/23 21:55:46 WARN GeneralizedLinearRegressionModel: When family is not tweedie, use param link to specify link function. Setting param linkPower will take no effect.


+--------+--------+--------------------+
|sequence|features|          prediction|
+--------+--------+--------------------+
| [427.0]| [427.0]|3.5035507098772645E8|
| [428.0]| [428.0]|3.5118910239173305E8|
| [429.0]| [429.0]| 3.520251192340565E8|
| [430.0]| [430.0]| 3.528631262410984E8|
| [431.0]| [431.0]|3.5370312815050673E8|
| [432.0]| [432.0]|3.5454512971121264E8|
| [433.0]| [433.0]| 3.553891356834472E8|
| [434.0]| [434.0]| 3.562351508387784E8|
| [435.0]| [435.0]|3.5708317996012807E8|
| [436.0]| [436.0]|  3.57933227841809E8|
| [437.0]| [437.0]| 3.587852992895419E8|
| [438.0]| [438.0]|3.5963939912049145E8|
| [439.0]| [439.0]| 3.604955321632898E8|
| [440.0]| [440.0]| 3.613537032580599E8|
| [441.0]| [441.0]|  3.62213917256452E8|
| [442.0]| [442.0]| 3.630761790216607E8|
| [443.0]| [443.0]|3.6394049342846256E8|
| [444.0]| [444.0]|3.6480686536323375E8|
| [445.0]| [445.0]|3.6567529972398776E8|
| [446.0]| [446.0]| 3.665458014203927E8|
+--------+--------+--------------------+
only showing top

In [108]:
from pyspark.sql.functions import  date_format,col, expr

# rename
purchase_future=purchase_future.withColumnRenamed("prediction","purchase")
redeem_future=redeem_future.withColumnRenamed("prediction","redeem")
purchase_future=purchase_future.select(col("features"),col("purchase"))
redeem_future=redeem_future.select(col("features"),col("redeem"))
result=purchase_future.join(redeem_future,on=["features"],how="inner")
result.show()


from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType

# 定义一个 UDF 将 features 列转换回 sequence 列
def extract_sequence(features):
    return int(features[0])
extract_sequence_udf = udf(extract_sequence, IntegerType())
def reverse_preprocess(df):
    df = df.withColumn("sequence", extract_sequence_udf(col("features")))
    df = df.drop("features")
    cols = ["sequence"] + [col for col in df.columns if col != "sequence"]
    df = df.select(cols)
    return df

df_reversed = reverse_preprocess(result)
# df_reversed.show()

df_reversed = df_reversed.withColumn("report_date", col("sequence")+lit(20140901-427))
df_reversed.show()
df_reversed=df_reversed.drop("report_date")
# df_reversed.show()

# 对sequence列，全部加上20140901-427，然后转换为INT
df_reversed = df_reversed.withColumn("sequence", col("sequence")+lit(20140901-427))
# df_reversed.show()

# rename aequence, data to int
df_reversed = df_reversed.withColumn("sequence", col("sequence").cast("int"))
df_reversed = df_reversed.withColumnRenamed("sequence", "report_date")
df_reversed = df_reversed.withColumn("purchase", col("purchase").cast("int"))
df_reversed = df_reversed.withColumn("redeem", col("redeem").cast("int"))
df_reversed.show()

# to csv
df_reversed.coalesce(1).write.mode("overwrite").csv("file:///home/siwenyu/桌面/Spark-lab4/res/task3_sqrt", header=False)


+--------+--------------------+--------------------+
|features|            purchase|              redeem|
+--------+--------------------+--------------------+
| [427.0]|3.5035507098772645E8|3.6150965247312015E8|
| [428.0]|3.5118910239173305E8|3.6292864996222806E8|
| [429.0]| 3.520251192340565E8| 3.643532172994987E8|
| [430.0]| 3.528631262410984E8| 3.657833763476982E8|
| [431.0]|3.5370312815050673E8|3.6721914905540866E8|
| [432.0]|3.5454512971121264E8| 3.686605574573606E8|
| [433.0]| 3.553891356834472E8|3.7010762367477936E8|
| [434.0]| 3.562351508387784E8|3.7156036991571903E8|
| [435.0]|3.5708317996012807E8| 3.730188184754048E8|
| [436.0]|  3.57933227841809E8|3.7448299173657644E8|
| [437.0]| 3.587852992895419E8| 3.759529121698264E8|
| [438.0]|3.5963939912049145E8| 3.774286023339527E8|
| [439.0]| 3.604955321632898E8| 3.789100848762998E8|
| [440.0]| 3.613537032580599E8| 3.803973825331089E8|
| [441.0]|  3.62213917256452E8| 3.818905181298613E8|
| [442.0]| 3.630761790216607E8| 3.833895145816

+-----------+---------+---------+
|report_date| purchase|   redeem|
+-----------+---------+---------+
|   20140901|350355070|361509652|
|   20140902|351189102|362928649|
|   20140903|352025119|364353217|
|   20140904|352863126|365783376|
|   20140905|353703128|367219149|
|   20140906|354545129|368660557|
|   20140907|355389135|370107623|
|   20140908|356235150|371560369|
|   20140909|357083179|373018818|
|   20140910|357933227|374482991|
|   20140911|358785299|375952912|
|   20140912|359639399|377428602|
|   20140913|360495532|378910084|
|   20140914|361353703|380397382|
|   20140915|362213917|381890518|
|   20140916|363076179|383389514|
|   20140917|363940493|384894394|
|   20140918|364806865|386405182|
|   20140919|365675299|387921899|
|   20140920|366545801|389444570|
+-----------+---------+---------+
only showing top 20 rows

